# Locqueneux Timéo, Le Rossignol Pierre-Jean, Bodin Titouan, Vandemeulebroucke Sacha

In [589]:
%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from IPython.display import Image

# INITIALISATION

In [591]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()
walletSacha = Wallet()
walletTiméo = Wallet()


pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)




# Smart contract importation

In [592]:
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from CollectionNFT import Collection
from certificate import Certificate
import inspect


reset_blockchain()


# collection : string du smart contract
collection = inspect.getsource(Collection)
dummySmartContractDefinition = SmartContractDefinition(walletAlice.publicKey, collection)
walletAlice.sign(dummySmartContractDefinition)

collection_nft = dummySmartContractDefinition.instantiate_collection(8)
node1.new_certificate(dummySmartContractDefinition)





Users can “mint” tokens under certain conditions :  
- Once the limit of N has been reached, there is no minting allowed anymore. 
- There should be a “mint” period. The owner of the collection can open the collection

In [593]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [node1.blockchain.get_latest_block().indexInBlockchain,2]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    [node1.blockchain.get_latest_block().indexInBlockchain]
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

Minting is open
Minted !
ownership changed


# Mint period with number of blocks

In [594]:

for i in range(15):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
walletAlice.publicKey,
dummySmartContractDefinition.hash(),
'mint_token',
[node1.blockchain.get_latest_block().indexInBlockchain]
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


Minting is closed


# Limit of N tokens

In [595]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [node1.blockchain.get_latest_block().indexInBlockchain,10]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)



    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    [node1.blockchain.get_latest_block().indexInBlockchain]
)

for i in range (10):
    walletAlice.sign(mint_token_SmartContractWritingOperation)
    mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
    node1.new_certificate(mint_token_SmartContractWritingOperation)


    for i in range(4):
        certificate = Certificate(walletAlice.publicKey)
        walletAlice.sign(certificate)
        node1.new_certificate(certificate)


Minting is open
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Limit of tokens reached
ownership changed
Limit of tokens reached
ownership changed
Limit of tokens reached
ownership changed


# Users can exchange ownership of their tokens

In [596]:
change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'change_ownership',
    [1,walletTiméo.publicKey]
)

walletAlice.sign(change_ownership_SmartContractWritingOperation)
change_ownership_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(change_ownership_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

ownership changed


In [597]:
def get_token_owner(token_number, smart_contract, node):
    smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node.blockchain,
    smart_contract.hash())
    print('\n','token number ', token_number, ' has public key :', smartContractObject.tokens[f'Token No{token_number}']['owner'][256:272])

get_token_owner(1, dummySmartContractDefinition, node1)
print("Timéo's wallet :      ",walletTiméo.publicKey[256:272])

change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
    walletTiméo.publicKey,
    dummySmartContractDefinition.hash(),
    'change_ownership',
    [1,walletSacha.publicKey]
)
walletTiméo.sign(change_ownership_SmartContractWritingOperation)
change_ownership_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(change_ownership_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)


get_token_owner(1, dummySmartContractDefinition, node1)
print("Sacha's wallet :    ",walletSacha.publicKey[256:272])

Minting is open
Minted !
ownership changed
Minting is closed
Minting is open
Minted !
ownership changed
ownership changed

 token number  1  has public key : 1975e1efe0bf274f
Timéo's wallet :       1975e1efe0bf274f
ownership changed
Minting is open
Minted !
ownership changed
Minting is closed
Minting is open
Minted !
ownership changed
ownership changed
ownership changed

 token number  1  has public key : eac9cd2f9c9d4e05
Sacha's wallet :     eac9cd2f9c9d4e05


# Create a collection of N tokens that all have a distinguishable identifier

In [598]:
print(collection_nft.tokens)
for token in collection_nft.tokens:
    print(token)

{'Token No0': {'name': 'pikachu', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric'}, 'Token No1': {'name': 'bulbasaur', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png', 'description': 'grass', 'owner': '30820122300d06092a864886f70d01010105000382010f003082010a02820101009f7f3ad01fb72384f79a7b1f6cc4d82dd4e35899a413e597cff39a970894769681d20294c42b436eecf53d7b18c82fff34db29051d9ad0b03561625676856d0d2ddf7cb9c6af1b33f493a86b5c36aeda0e3204c64fc57762edf3a9da3de265eac9cd2f9c9d4e058888bf596936f70c05fdc433db997e81e05a5df358508ac709bdc8e6593f04d957f99eeec33e33813db0da070dabe8d39f8c40e1ccaa4e5b3f94da874f23513e7f8c5011696657b296d1f662579effc8a30d8371bd1813a4814ad0880840cf1cdd53ac958cdaf2b9c476fcf86e26a9b41c96d24439f7079afd10cde2e67d33de0465748a523c2c7930a96d35d44ee985deca4584520dd27c330203010001'}, 'Token No2': {'name': 'charmander', 'image_link': 'https://raw.githubusercontent.com

Each token must be linked with a unique image or a description of a game item

In [599]:
collection_nft.tokens['Token No3']

{'name': 'squirtle',
 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png',
 'description': 'water',
 'owner': '30820122300d06092a864886f70d01010105000382010f003082010a0282010100b224e8dd2495122da56988dbcd8ea241d67448f6351afe04b316a20d9e6e2b5db74324997c923f3ad12c2a6d605422ff5632596c470de73c7078d1ff3b2bb9136a1329be22df4a2671d2569752fe694dd9d803af1c9f96b7fc17e0a0b869c6e08f32225644f86932ce87e824be83a4a1bfe3a50c6c1f0859800f16abccb994727224b759cd8f961a6c16a94d6d08f6e47917fa79eb9b359e52d71af9da861d88b3fd159515ebded5704c94ccfda4064c36aba4a05e82f0e3c6960359149c658d299198ee78655a2c2c89e2ec68c9e5309e74a3292372606878f91d210b4111a42f3c744aa51ca85e20e3768b28e75387f319deb14af5c9b509a96f18ee8311b90203010001'}

In [ ]:
Image(url=collection_nft.tokens['Token No3']['image_link'])
Image(url=collection_nft.tokens['Token No1']['image_link'])
Image(url=collection_nft.tokens['Token No2']['image_link'])
Image(url=collection_nft.tokens['Token No4']['image_link'])
